Recurrent Neural networks
=====

### RNN  

<img src ="imgs/rnn.png" width="20%">

A recurrent neural network (RNN) is a class of artificial neural network where connections between units form a directed cycle. This creates an internal state of the network which allows it to exhibit dynamic temporal behavior.

In [1]:
import keras
output_dim = 100

Using TensorFlow backend.


In [2]:
keras.layers.recurrent.SimpleRNN(output_dim, 
                                 init='glorot_uniform', inner_init='orthogonal', activation='tanh', 
                                 W_regularizer=None, U_regularizer=None, b_regularizer=None, 
                                 dropout_W=0.0, dropout_U=0.0)

#### Backprop Through time  

Contrary to feed-forward neural networks, the RNN is characterized by the ability of encoding longer past information, thus very suitable for sequential models. The BPTT extends the ordinary BP algorithm to suit the recurrent neural
architecture.

<img src ="imgs/rnn2.png" width="45%">

In [3]:
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd
import theano
import theano.tensor as T
import keras 
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing import image
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import imdb
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from keras.layers.core import Activation, TimeDistributedDense, RepeatVector
from keras.callbacks import EarlyStopping, ModelCheckpoint

import tensorflow as tf

tf.python.control_flow_ops = tf

/home/ubuntu/anaconda3/envs/deep-learning/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### IMDB sentiment classification task

This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. 

IMDB provided a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. 

There is additional unlabeled data for use as well. Raw text and already processed bag of words formats are provided. 

http://ai.stanford.edu/~amaas/data/sentiment/

#### Data Preparation - IMDB

In [5]:
max_features = 20000
maxlen = 100  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print("Loading data...")
# (X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features, test_split=0.2)
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Example:')
print(X_train[:1])

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

max_epoch = 50

Loading data...
25000 train sequences
25000 test sequences
Example:
[ [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 1

In [6]:
X_train.shape

(25000, 100)

#### Model building 

In [7]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SimpleRNN(128))  
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy"])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(X_test, y_test))
acc , loss = model.evaluate(X_test, y_test, batch_size=batch_size)
print()
print('Test accuracy:', acc)
print('Test loss:', loss)

Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/1
24992/25000 [============================>.] - ETA: 0s
Test accuracy: 0.602803128452
Test loss: 0.65844


### LSTM  

A LSTM network is an artificial neural network that contains LSTM blocks instead of, or in addition to, regular network units. A LSTM block may be described as a "smart" network unit that can remember a value for an arbitrary length of time. 

Unlike traditional RNNs, an Long short-term memory network is well-suited to learn from experience to classify, process and predict time series when there are very long time lags of unknown size between important events.

<img src ="imgs/gru.png" width="60%">

In [8]:
keras.layers.recurrent.LSTM(output_dim, init='glorot_uniform', inner_init='orthogonal', 
                            forget_bias_init='one', activation='tanh', 
                            inner_activation='hard_sigmoid', 
                            W_regularizer=None, U_regularizer=None, b_regularizer=None, 
                            dropout_W=0.0, dropout_U=0.0)

### GRU  

Gated recurrent units are a gating mechanism in recurrent neural networks. 

Much similar to the LSTMs, they have fewer parameters than LSTM, as they lack an output gate.

In [9]:
keras.layers.recurrent.GRU(output_dim, init='glorot_uniform', inner_init='orthogonal', 
                           activation='tanh', inner_activation='hard_sigmoid', 
                           W_regularizer=None, U_regularizer=None, b_regularizer=None, 
                           dropout_W=0.0, dropout_U=0.0)

### Your Turn! - Hands on Rnn

In [10]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))

# Play with those! try and get better results!
model.add(SimpleRNN(128))  
#model.add(GRU(128))  
#model.add(LSTM(128))  

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=["accuracy"])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, 
          nb_epoch=max_epoch, validation_data=(X_test, y_test))
acc , loss = model.evaluate(X_test, y_test, batch_size=batch_size)
print()
print('Test accuracy:', acc)
print('Test loss:', loss)

Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/50
25000/25000 [==============================] - 67s - loss: 0.6788 - acc: 0.5844 - val_loss: 0.6298 - val_acc: 0.6260
Epoch 2/50
25000/25000 [==============================] - 68s - loss: 0.5956 - acc: 0.6794 - val_loss: 0.6144 - val_acc: 0.6508
Epoch 3/50
25000/25000 [==============================] - 67s - loss: 0.4994 - acc: 0.7652 - val_loss: 0.6644 - val_acc: 0.5839
Epoch 4/50
25000/25000 [==============================] - 68s - loss: 0.5675 - acc: 0.7043 - val_loss: 0.5865 - val_acc: 0.7020
Epoch 5/50
25000/25000 [==============================] - 70s - loss: 0.4869 - acc: 0.7755 - val_loss: 0.5547 - val_acc: 0.7499
Epoch 6/50
25000/25000 [==============================] - 107s - loss: 0.5271 - acc: 0.7411 - val_loss: 0.6187 - val_acc: 0.6920
Epoch 7/50
25000/25000 [==============================] - 115s - loss: 0.4682 - acc: 0.7920 - val_loss: 0.5909 - val_acc: 0.7024
Epoch 8/50
25000/25000 [====

In [11]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))

# Play with those! try and get better results!
# model.add(SimpleRNN(128))  
model.add(GRU(128))  
# model.add(LSTM(128))  

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=["accuracy"])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, 
          nb_epoch=max_epoch, validation_data=(X_test, y_test))
acc , loss = model.evaluate(X_test, y_test, batch_size=batch_size)
print()
print('Test accuracy:', acc)
print('Test loss:', loss)

Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/50
25000/25000 [==============================] - 202s - loss: 0.4200 - acc: 0.8056 - val_loss: 0.3351 - val_acc: 0.8532
Epoch 2/50
25000/25000 [==============================] - 201s - loss: 0.2216 - acc: 0.9145 - val_loss: 0.3506 - val_acc: 0.8560
Epoch 3/50
25000/25000 [==============================] - 199s - loss: 0.1111 - acc: 0.9619 - val_loss: 0.3902 - val_acc: 0.8461
Epoch 4/50
25000/25000 [==============================] - 198s - loss: 0.0564 - acc: 0.9810 - val_loss: 0.5727 - val_acc: 0.8374
Epoch 5/50
25000/25000 [==============================] - 199s - loss: 0.0307 - acc: 0.9902 - val_loss: 0.7406 - val_acc: 0.8356
Epoch 6/50
25000/25000 [==============================] - 196s - loss: 0.0203 - acc: 0.9940 - val_loss: 0.7308 - val_acc: 0.8356
Epoch 7/50
25000/25000 [==============================] - 197s - loss: 0.0105 - acc: 0.9967 - val_loss: 0.9419 - val_acc: 0.8248
Epoch 8/50
25000/25000 

In [12]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))

# Play with those! try and get better results!
# model.add(SimpleRNN(128))  
# model.add(GRU(128))  
model.add(LSTM(128))  

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=["accuracy"])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, 
          nb_epoch=max_epoch, validation_data=(X_test, y_test))
acc , loss = model.evaluate(X_test, y_test, batch_size=batch_size)
print()
print('Test accuracy:', acc)
print('Test loss:', loss)

Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/50
25000/25000 [==============================] - 227s - loss: 0.4267 - acc: 0.8055 - val_loss: 0.3564 - val_acc: 0.8474
Epoch 2/50
25000/25000 [==============================] - 223s - loss: 0.2367 - acc: 0.9107 - val_loss: 0.4825 - val_acc: 0.8349
Epoch 3/50
25000/25000 [==============================] - 219s - loss: 0.1385 - acc: 0.9496 - val_loss: 0.4615 - val_acc: 0.8372
Epoch 4/50
25000/25000 [==============================] - 220s - loss: 0.0794 - acc: 0.9730 - val_loss: 0.5870 - val_acc: 0.8321
Epoch 5/50
25000/25000 [==============================] - 223s - loss: 0.0543 - acc: 0.9832 - val_loss: 0.6056 - val_acc: 0.8249
Epoch 6/50
25000/25000 [==============================] - 219s - loss: 0.0347 - acc: 0.9885 - val_loss: 0.7454 - val_acc: 0.8268
Epoch 7/50
25000/25000 [==============================] - 228s - loss: 0.0304 - acc: 0.9904 - val_loss: 0.7272 - val_acc: 0.8207
Epoch 8/50
25000/25000 